# APACHE CASSANDRA TUTORIAL
Here are some references to learn more about Apache Cassandra:

Apache Cassandra Data Modeling and Query Best Practices:

<https://www.red-gate.com/simple-talk/sql/nosql-databases/apache-cassandra-data-modeling-and-query-best-practices/>

Top 5 reasons to use the Apache Cassandra Database:

<https://towardsdatascience.com/top-5-reasons-to-use-the-apache-cassandra-database-d541c6448557>

Introduction to Apache Cassandra:

<https://www.geeksforgeeks.org/introduction-to-apache-cassandra/>

Difference between partition key, composite key and clustering key in Cassandra

<https://stackoverflow.com/questions/24949676/difference-between-partition-key-composite-key-and-clustering-key-in-cassandra/24953331#24953331>

# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [4]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [5]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/50526/Desktop/personal/udacity/data-engineering/cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [6]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [7]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Query with the Apache Cassandra

## The event_datafile_new.csv contains the data that will be stored in the databased and it's composed by the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the event_datafile_new.csv after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [ ]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [17]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS cassandra_assignment 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [18]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('cassandra_assignment')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. With Apache Cassandra you model the database tables on the queries you want to run.

## Query 1: Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
Since we have to filter on sessionId and itemInSession, and their combination is unique, we can use these two columns as PRIMARY KEY

In [20]:
query = "CREATE TABLE IF NOT EXISTS songs_by_session_and_item "
query = query + "(artist text, song text, length float, sessionId int, itemInSession int, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [31]:
query1 = "SELECT artist, song, length FROM songs_by_session_and_item WHERE sessionId = %s AND itemInSession = %s"

In [32]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO songs_by_session_and_item (artist, song, length, sessionId, itemInSession) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

In [33]:
## Add in the SELECT statement to verify the data was entered into the table
try:
    rows = session.execute(query1, (338, 4))
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
Since we want to filter on userid and sessionid, the two columns will be in the PRIMARY KEY, given we want the result to be sorted by itemInSession we need to add it as CLUSTERING COLUMN, to make sure this happens we need set userId, sessionId) as a COMPOSITE KEY.

In [44]:
# This query uses a COMPOSITE KEY
query = "CREATE TABLE IF NOT EXISTS songs_and_users_by_userid_and_session "
query = query + """(
    userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text,
    PRIMARY KEY ((userId, sessionId), itemInSession)
    )
    """
try:
    session.execute(query)
except Exception as e:
    print(e)

In [53]:
query2 = "SELECT artist, song, firstName, lastName FROM songs_and_users_by_userid_and_session WHERE userId = %s AND sessionId = %s"

In [46]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO songs_and_users_by_userid_and_session (userId, sessionId, itemInSession, artist, song, firstName, lastName) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [56]:
try:
    rows = session.execute(query2, (10, 182))
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
Since we do not require to display the same name twice in case a song was listened two times by the same user, the most efficient PRIMARY KEYs in this case are song and userId.

In [61]:
# This query uses userId as key to keep the PRIMARY KEY combination as simple as possible for maximum efficiency.
query = "CREATE TABLE IF NOT EXISTS user_by_song_name "
query = query + """(
    firstName song text, userId int, firstName text, lastName text,
    PRIMARY KEY (song, userId)
    )
    """
try:
    session.execute(query)
except Exception as e:
    print(e)

In [84]:
query3 = "SELECT firstName, lastName FROM user_by_song_name WHERE song = %s"

In [62]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_by_song_name (song, userId, firstName, lastName) "
        query = query + "VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [85]:
try:
    rows = session.execute(query3, ("All Hands Against His Own", ))
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Sara Johnson
Jacqueline Lynch
Tegan Levine


### Drop the tables before closing out the sessions

In [90]:
try:
    rows = session.execute("DROP TABLE IF EXISTS music_library_query1")
except Exception as e:
    print(e)
try:
    rows = session.execute("DROP TABLE IF EXISTS music_library_query2")
except Exception as e:
    print(e)
try:
    rows = session.execute("DROP TABLE IF EXISTS music_library_query3")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [91]:
session.shutdown()
cluster.shutdown()